## Data Preprocessing

In [1]:
# Import relevant libraries
import numpy as np
import pandas as pd

In [2]:
# Load raw data
raw_data = pd.read_csv('Absenteeism-data.csv')
raw_data.head(30)

,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,11,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2
5,3,23,10/07/2015,179,51,38,239.554,31,1,0,0,2
6,10,22,17/07/2015,361,52,28,239.554,27,1,1,4,8
7,20,23,24/07/2015,260,50,36,239.554,23,1,4,0,4
8,14,19,06/07/2015,155,12,34,239.554,25,1,2,0,40
9,1,22,13/07/2015,235,11,37,239.554,29,3,1,1,8


## Reason for absence
(Reasons 1-21 are registered in the International Classification of Diseases (ICD), Reasons 22-28 are not)
1. Certain infectious and parasite disease
2. Neoplasm
3. Disease of the blood and blood-forming organs and certain disorders involving the immune mechanism
4. Endocrine, nutritional and metabolic disease
5. Mental and behavioural disorders
6. Diseases of the nervous system
7. Diseases of the eye and adnexa
8. Diseases of the ear and mastoid process
9. Diseases of circulatory system
10. Diseases of the respiratory system
11. Diseases of the digestive system
12. Diseases of the skin and subcutaneous tissue
13. Diseases of the musculoskeletal system and connective tissue
14. Diseases of the genitourinary system
15. Pregnancy, childbirth and the puerperium
16. Certain conditions originating in the perinatal period
17. Congenital malformations, deformations and chromosomal abnormalities
18. Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified
19. Injury, poisoning and certain other consequences of external causes
20. External causes of morbidity and mortality
21. Factors influencing health status and contact with health services
22. Patient follow-up
23. Medical consultation
24. Blood donation
25. Laboratory examination
26. Unjustified absence
27. Physiotherapy
28. Dental consultation

In [3]:
# Create a checkpoint
data = raw_data.copy()

# Drop ID
data = data.drop(['ID'], axis=1)

# Create dummy variables from Reasons for Absence
reason_columns = pd.get_dummies(data['Reason for Absence'], drop_first=True)

# To confirm there is just one reason for absence
# reason_columns['check'] = reason_columns.sum(axis=1)
# reason_columns = reason_columns.drop(['check'],axis=1)
# reason_columns 

# Drop Reason for Absence column
data = data.drop(['Reason for Absence'], axis=1)
# data.head()

# Group Reason for Absence
reason_type_1 = reason_columns.loc[:, 1:14].max(axis=1)
reason_type_2 = reason_columns.loc[:, 15:17].max(axis=1)
reason_type_3 = reason_columns.loc[:, 18:21].max(axis=1)
reason_type_4 = reason_columns.loc[:, 22:].max(axis=1)

# Concatenate column values
data = pd.concat([data, reason_type_1, reason_type_2, reason_type_3, reason_type_4], axis = 1)

# Column values
column_names = ['Date', 'Transportation Expense', 'Distance to Work', 'Age',
                'Daily Work Load Average', 'Body Mass Index', 'Education',
                'Children', 'Pets', 'Absenteeism Time in Hours', 'Reason_1', 'Reason_2', 'Reason_3', 'Reason_4']
data.columns = column_names

# Reorder column values
column_names_reordered = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 
                          'Date', 'Transportation Expense', 'Distance to Work', 'Age',
                          'Daily Work Load Average', 'Body Mass Index', 'Education',
                          'Children', 'Pets', 'Absenteeism Time in Hours']

data = data[column_names_reordered]

# Data reason modified checkpoint
data_reason_modified = data.copy()

# Format date
data_reason_modified['Date'] = pd.to_datetime(data_reason_modified['Date'], format = '%d/%m/%Y')

# Extract the Month value
list_months = []

for i in range(data_reason_modified.shape[0]):
    list_months.append(data_reason_modified['Date'][i].month)

data_reason_modified['Month Value'] = list_months

def date_to_weekday(date_value):
    return date_value.weekday()

data_reason_modified['Day of the Week'] = data_reason_modified['Date'].apply(date_to_weekday)

# Drop the Date column
data_reason_modified = data_reason_modified.drop(['Date'], axis = 1)
# data_reason_modified.head()

# Update column names
column_names_upd = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value', 'Day of the Week',
       'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education', 'Children',
       'Pets', 'Absenteeism Time in Hours']

data_reason_modified = data_reason_modified[column_names_upd]

# Data reason date modified checkpoint
data_reason_date_modified = data_reason_modified.copy()

# Education mapping
# data_reason_date_modified['Education'].unique()
# data_reason_date_modified['Education'].value_counts()
data_reason_date_modified['Education'] = data_reason_date_modified['Education'].map({1:0, 2:1, 3:1, 4:1})
# data_reason_date_modified['Education'].unique()

# Data preprocessed checkpoint
data_preprocessed = data_reason_date_modified.copy()

# Save data preprocessed as csv for modelling
data_preprocessed.to_csv('Absenteeism_preprocessed.csv', index = False)